In [1]:
!pip install sentence-transformers faiss-cpu rank-bm25 fastapi uvicorn httpx rapidfuzz sacrebleu rouge-score sqlalchemy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 11.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=7661a154d35eff05d8ebcdccebe2141934934766e75f4f25c9ba7653e773b4cb
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [2]:
import pandas as pd
import re
import json

df1 = pd.read_csv('/content/Natural-Questions-Base.csv')
df2 = pd.read_csv('/content/Natural-Questions-Filtered.csv')
df = pd.concat([df1, df2], ignore_index=True).drop_duplicates()

print("Dataset shape:", df.shape)

Dataset shape: (393585, 3)


In [3]:
def clean(text):
    if not isinstance(text, str):
        return ""
    text = text.replace("\n", " ").replace("\r", " ")
    return re.sub(r"\s+", " ", text).strip()


def chunk(text, max_words=160, overlap=30):
    words = text.split()
    chunks = []
    step = max_words - overlap

    for i in range(0, len(words), step):
        chunks.append(" ".join(words[i:i+max_words]))

    return chunks

In [4]:
def question_type(q):
    m = re.match(r"(who|what|when|where|why|how)", q.lower())
    return m.group(1) if m else "other"


def domain(q):
    q = q.lower()
    if any(x in q for x in ["movie", "actor", "film"]):
        return "entertainment"
    if any(x in q for x in ["city", "country", "capital"]):
        return "geography"
    if any(x in q for x in ["war", "president", "empire"]):
        return "history"
    return "general"


def difficulty(short_answers, long_answer):
    if short_answers and len(long_answer) < 300:
        return "easy"
    if short_answers and len(long_answer) >= 300:
        return "medium"
    return "hard"

In [5]:
records = []

for i, row in df.iterrows():
    q = clean(row.get("question", ""))
    la = clean(row.get("long_answers", ""))
    sa = clean(row.get("short_answers", ""))

    sa_list = [x.strip() for x in re.split(r"[;,|]", sa) if x.strip()]

    rec = {
        "id": f"nq_{i}",
        "question": q,
        "short_answers": sa_list,
        "chunks": chunk(la),
        "metadata": {
            "type": question_type(q),
            "domain": domain(q),
            "difficulty": difficulty(sa_list, la)
        }
    }

    records.append(rec)

print("Processed records:", len(records))

Processed records: 393585


In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

texts = []
meta = []

for r in records:
    for c in r["chunks"]:
        texts.append(c)
        meta.append({
            "id": r["id"],
            "question": r["question"],
            "chunk": c,
            "metadata": r["metadata"]
        })

print("Embedding", len(texts), "chunks...")
vectors = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

dim = vectors.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(vectors.astype("float32"))

print("FAISS ready.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Embedding 629689 chunks...
FAISS ready.


In [7]:
from rank_bm25 import BM25Okapi

tokenized = [re.findall(r"\w+", t.lower()) for t in texts]
bm25 = BM25Okapi(tokenized)

In [8]:
def normalize_query(q):
    q = q.lower()
    return re.sub(r"\s+", " ", q).strip()

def expand_query(q):
    expansions = {
        "usa": "united states america",
        "ai": "artificial intelligence"
    }
    for k, v in expansions.items():
        if k in q:
            q += " " + v
    return q

In [9]:
def retrieve(query, top_k=5):
    query = normalize_query(query)
    query = expand_query(query)

    qv = model.encode([query], normalize_embeddings=True)
    scores, idxs = index.search(qv.astype("float32"), top_k)

    tokens = re.findall(r"\w+", query.lower())
    bm25_scores = bm25.get_scores(tokens)
    max_bm = max(bm25_scores) if max(bm25_scores) > 0 else 1

    combined = []
    for idx, score in zip(idxs[0], scores[0]):
        hybrid_score = 0.7 * score + 0.3 * (bm25_scores[idx] / max_bm)
        combined.append((idx, hybrid_score))

    combined.sort(key=lambda x: x[1], reverse=True)

    return [meta[i] for i, _ in combined[:top_k]]

In [10]:
import requests
import os

def generate_answer(question, contexts):
    context_text = "\n\n".join([c["chunk"] for c in contexts])

    prompt = f"""
    Answer strictly using the context below.

    Context:
    {context_text}

    Question:
    {question}
    """

    response = requests.post(
        "https://api.groq.com/v1/chat/completions",
        headers={"Authorization": f"Bearer {os.getenv('GROQ_API_KEY')}"},
        json={
            "model": "llama3-70b-8192",
            "messages": [{"role": "user", "content": prompt}]
        }
    )

    return response.json()

In [11]:
from functools import lru_cache

@lru_cache(maxsize=500)
def cached_retrieve(query):
    return retrieve(query)

In [12]:
from rapidfuzz import fuzz
import numpy as np
import sacrebleu
from rouge_score import rouge_scorer


def precision_at_k(contexts, gold, k=5):
    if not gold:
        return 0

    hits = 0
    for c in contexts[:k]:
        for g in gold:
            if fuzz.partial_ratio(c["chunk"].lower(), g.lower()) > 80:
                hits += 1
                break

    return hits / k


def bleu_rouge(hypothesis, gold):
    if not gold:
        return None

    bleu = sacrebleu.corpus_bleu([hypothesis], [gold]).score
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    rougeL = max(scorer.score(hypothesis, g)["rougeL"].fmeasure for g in gold)

    return {"BLEU": bleu, "ROUGE-L": rougeL}

In [13]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class QuestionRequest(BaseModel):
    question: str
    top_k: int = 5


@app.post("/ask-question")
def ask_question(req: QuestionRequest):
    docs = retrieve(req.question, req.top_k)
    answer = generate_answer(req.question, docs)
    return {"answer": answer}


@app.get("/health")
def health():
    return {"status": "ok"}

In [18]:
%%writefile main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"message": "API working"}

Writing main.py


In [20]:
!pip install pyngrok

In [39]:
public_url = ngrok.connect(8000)
print("🚀 Public FastAPI URL:", public_url)
!uvicorn main:app --host 0.0.0.0 --port 8000 --reload --log-level info

🚀 Public FastAPI URL: NgrokTunnel: "https://de9c-34-143-176-202.ngrok-free.app" -> "http://localhost:8000"
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [27562] using WatchFiles
INFO:     Started server process [27568]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     41.36.16.221:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     41.36.16.221:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [27568]
INFO:     Stopping reloader process [27562]


In [38]:
from pyngrok import ngrok

ngrok.set_auth_token("32lBaLkWSAbaa3pN0QOLq7UH0xe_uoJ6vrzVhwp794o59qM2")

public_url = ngrok.connect(8000)
print(public_url)

NgrokTunnel: "https://a7ef-34-143-176-202.ngrok-free.app" -> "http://localhost:8000"


In [31]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [24204] using WatchFiles
INFO:     Started server process [24206]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     41.36.16.221:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     41.36.16.221:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     41.36.16.221:0 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [24206]
INFO:     Stopping reloader process [24204]


In [37]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/predict")
def predict(text: str):
    return {"received_text": text}

In [46]:
query = "Who is Nikola Tesla?"
docs = retrieve(query, top_k=3)

for d in docs:
    print("\n--- Retrieved Chunk ---")
    print(d["chunk"])


--- Retrieved Chunk ---
<Li> Nicola Peltz as Katara </Li>

--- Retrieved Chunk ---
<Ul> <Li> Nicola Peltz as Katara </Li> </Ul>

--- Retrieved Chunk ---
Natasha Lynne as Nicky Nichols, inmate
